<a href="https://colab.research.google.com/github/ralph27/ZAKA-hands-on/blob/master/Data_Preparation_for_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Image Data Preparation
© 2024, Zaka AI, Inc. All Rights Reserved.

---

**Objective:** This exercise explores on the Keras library's role in computer vision. It shows how to load and convert images with Keras, how to use the MNIST handwritten image classification dataset and how to employ the ImageDataGenerator class.

Let's start by getting our dataset

In [ ]:
# clone github repo
!git clone https://github.com/zaka-ai/computer-vision-course.git

Cloning into 'computer-vision-course'...
remote: Enumerating objects: 2118, done.
remote: Counting objects: 100% (3/3), done.
remote: Compressing objects: 100% (3/3), done.
remote: Total 2118 (delta 0), reused 0 (delta 0), pack-reused 2115
Receiving objects: 100% (2118/2118), 51.06 MiB | 20.87 MiB/s, done.
Resolving deltas: 100% (34/34), done.


## How to Load an Image with Keras

Keras provides the `load_img()` function for loading an image from file as a PIL image object.

The example below loads the Bondi Beach photograph from file as a PIL image and reports details about the loaded image.

In [ ]:
# example of loading an image with the Keras API
from tensorflow.keras.preprocessing.image import load_img

# load the image
img_path = "computer-vision-course/deep_learning/data/image1.jpg"
img = load_img(img_path)

# report details about the image
print(type(img))
print(img.format)
print(img.mode)
print(img.size)

<class 'PIL.JpegImagePlugin.JpegImageFile'>
JPEG
RGB
(640, 360)


## How to Convert an Image With Keras

Keras provides the `img_to_array()` function for converting a loaded image in PIL format into a NumPy array for use with deep learning models. The API also provides the `array_to_img()` function that can be used for converting a NumPy array of pixel data into a PIL image. This can be useful if the pixel data is modified in array format because it can be saved or viewed.

The example below loads the test image, converts it to a NumPy array, and then converts it back into a PIL image.

In [ ]:
# example of converting an image with the Keras API
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.preprocessing.image import array_to_img

# load the image
img_path = "computer-vision-course/deep_learning/data/image1.jpg"
img = load_img(img_path)
print(type(img))

# convert to numpy array
img_array = img_to_array(img)
print(img_array.dtype)
print(img_array.shape)

# convert back to image
img_pil = array_to_img(img_array)
print(type(img))

<class 'PIL.JpegImagePlugin.JpegImageFile'>
float32
(360, 640, 3)
<class 'PIL.JpegImagePlugin.JpegImageFile'>


## ImageDataGenerator Class

The ImageDataGenerator provides a suite of techniques for dealing with images such as scaling image pixels, applying pixel centering, applying augmentations, and many more!

### MNIST Handwritten Image Classification Dataset

In [ ]:
from tensorflow.keras.datasets import mnist

(trainX, trainY), (testX, testY) = mnist.load_data()

# summarize dataset shape
print("Train", trainX.shape, trainY.shape)
print("Test", testX.shape, testY.shape)

# summarize pixel values
print("Train", trainX.min(), trainX.max(), trainX.mean(), trainX.std())
print("Test", testX.min(), testX.max(), testX.mean(), testX.std())

11490434/11490434 [==============================] - 1s 0us/step
Train (60000, 28, 28) (60000,)
Test (10000, 28, 28) (10000,)
Train 0 255 33.318421449829934 78.56748998339798
Test 0 255 33.791224489795916 79.17246322228644


### How to Normalize Images With ImageDataGenerator

**Reshape dataset:** the MNIST dataset is loaded with number of samples, number of pixels in width and height but not number of channels. This is why we make sure that the ImageDataGenerator class and Keras functions receive datasets having the number of samples, the size and the depth all together.

In [ ]:
# reshape dataset to have a single channel because the ImageDataGenerator expects a numpy array of rank 4

# trainX: (60000, 28, 28) => (60000, 28, 28, 1)
# testX:  (10000, 28, 28) => (10000, 28, 28, 1)

trainX = trainX.reshape((trainX.shape[0], trainX.shape[1], trainX.shape[2], 1))
testX = testX.reshape((testX.shape[0], trainX.shape[1], trainX.shape[2], 1))

In [ ]:
# summarize dataset shape
print("Train", trainX.shape, trainY.shape)
print("Test", testX.shape, testY.shape)

Train (60000, 28, 28, 1) (60000,)
Test (10000, 28, 28, 1) (10000,)


**Rescale:** Normalize all the pixel values of the dataset to an interval of 0 to 1 instead of having small weight values when training deep learning networks for example and very large pixel values by comparison.

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# confirm scale of pixels
print('Train min=%.3f, max=%.3f' % (trainX.min(), trainX.max()))
print('Test min=%.3f, max=%.3f' % (testX.min(), testX.max()))

# create generator (1.0/255.0)
datagen = ImageDataGenerator(rescale=1.0/255.0)



Train min=0.000, max=255.000
Test min=0.000, max=255.000


Next, iterators can be created using the generator for both the train and test datasets. We will use a batch size of 64. This means that each of the train and test datasets of images are divided into groups of 64 images that will then be scaled when returned from the iterator.

We can see how many batches there will be in one epoch, e.g. one pass through the training dataset, by printing the length of each iterator.

In [ ]:
# prepare iterators to scale images
train_iterator = datagen.flow(trainX, trainY, batch_size=64)
test_iterator = datagen.flow(testX, testY, batch_size=64)

# 60000 / 64 = 937.5 ~= 938

print('Batches train=%d, test=%d' % (len(train_iterator), len(test_iterator)))

# confirm the scaling works
batchX, batchy = train_iterator.next()
print('Batch shape=%s, min=%.3f, max=%.3f' % (batchX.shape, batchX.min(), batchX.max()))

Batches train=938, test=157
Batch shape=(64, 28, 28, 1), min=0.000, max=1.000


### How to Center Images With ImageDataGenerator
**Center pixel values:** Find the mean of the pixel values and make sure all the pixels are distributed around it as if it were the new zero and the remaining values are positive and negative.


In [ ]:
# report per-image mean
print('Means train=%.3f, test=%.3f' % (trainX.mean(), testX.mean()))

# create generator that centers pixel values
datagen = ImageDataGenerator(featurewise_center=True)

# calculate the mean on the training dataset
datagen.fit(trainX)
print('Data Generator Mean: %.3f' % datagen.mean)

# demonstrate effect on a single batch of samples
iterator = datagen.flow(trainX, trainY, batch_size=64)

# get a batch
batchX, batchy = iterator.next()

# mean pixel value in the batch
print(batchX.shape, batchX.mean())


Means train=33.318, test=33.791
Data Generator Mean: 33.318
(64, 28, 28, 1) -0.3496583


A better check would be to set the batch size to the size of the training dataset (e.g. 60,000 samples), retrieve one batch, then calculate the mean. It should be a very small value close to zero.

In [ ]:

# demonstrate effect on entire training dataset
iterator = datagen.flow(trainX, trainY, batch_size=len(trainX), shuffle=False)

# get a batch
batchX, batchy = iterator.next()

# mean pixel value in the batch
print(batchX.shape, batchX.mean())

#display an image
img2 = batchX[41]
print(img2.shape)
from google.colab.patches import cv2_imshow
cv2_imshow(img2)

(60000, 28, 28, 1) -1.9512918e-05
(28, 28, 1)


## TASK: Load dataset images progressively
In this task, we will code a Keras API set of instructions to progressively load data from disk.

In [ ]:
# import IDG from Keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# create the data generator with rescale and iterators
datagen = ImageDataGenerator(rescale = 1.0/255.0)
train_it = datagen.flow_from_directory("computer-vision-course/deep_learning/dataset/train", class_mode = "binary", batch_size = 2)
test_it = datagen.flow_from_directory("computer-vision-course/deep_learning/dataset/test", class_mode="binary", batch_size=1)
valid_it = datagen.flow_from_directory("computer-vision-course/deep_learning/dataset/validation", class_mode="binary", batch_size=1)

# get batches for training
batchX, batchY = train_it.next()
# get info on batches
print('Batch shape=%s, min=%.3f, max=%.3f' % (batchX.shape, batchX.min(), batchX.max()))

# testing batch
batchX, batchy = test_it.next()
# get info on batches
print('Batch shape=%s, min=%.3f, max=%.3f' % (batchX.shape, batchX.min(), batchX.max()))

# validation batch
batchX, batchy = valid_it.next()
# get info on batches
print('Batch shape=%s, min=%.3f, max=%.3f' % (batchX.shape, batchX.min(), batchX.max()))

Found 20 images belonging to 2 classes.
Found 4 images belonging to 2 classes.
Found 10 images belonging to 2 classes.
Batch shape=(2, 256, 256, 3), min=0.000, max=1.000
Batch shape=(1, 256, 256, 3), min=0.000, max=1.000
Batch shape=(1, 256, 256, 3), min=0.000, max=1.000
